In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def is_folder_readable(path):
    """
    Check if a folder is readable and contains images.
    """
    try:
        files = os.listdir(path)
        return True if files else False
    except Exception as e:
        return False

def list_unreadable_folders(base_dir):
    """
    List all folders in the base directory that are not readable.
    """
    folders = os.listdir(base_dir)
    unreadable_folders = []
    
    for folder in folders:
        folder_path = os.path.join(base_dir, folder)
        if not is_folder_readable(folder_path):
            unreadable_folders.append(folder_path)
    
    return unreadable_folders

def handle_unreadable_folders(unreadable_folders):
    """
    Ask user if they want to include unreadable folders and check them again.
    """
    resolved_folders = []
    
    for folder_path in unreadable_folders:
        print(f"Folder '{folder_path}' is not readable.")
        include = input("Do you want to try including this folder? (yes/no): ").strip().lower()
        
        if include == 'yes':
            new_path = input("Enter the new path for this folder: ").strip()
            while not is_folder_readable(new_path):
                print(f"Path '{new_path}' is still not valid. Please enter a new path.")
                new_path = input("Enter the new path for this folder: ").strip()
            resolved_folders.append(new_path)
        else:
            print(f"Skipping folder '{folder_path}'.")
    
    return resolved_folders

def copy_data(src_dir, dest_dir):
    """
    Copy data from source directory to destination directory.
    """
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)
    
    for folder in os.listdir(src_dir):
        src_folder_path = os.path.join(src_dir, folder)
        dest_folder_path = os.path.join(dest_dir, folder)
        if os.path.isdir(src_folder_path):
            shutil.copytree(src_folder_path, dest_folder_path, dirs_exist_ok=True)

def folder_contains_images(folder):
    """
    Check if a folder contains any images.
    """
    return any([file for file in os.listdir(folder) if file.endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp'))])

# Main execution
base_dir = input("Enter the path to the main dataset directory: ").strip()
train_dir = input("Enter the path for the train folder: ").strip()
test_dir = input("Enter the path for the test folder: ").strip()

# Check if train and test directories already contain images
train_images_exist = any([folder_contains_images(os.path.join(train_dir, d)) for d in os.listdir(train_dir)])
test_images_exist = any([folder_contains_images(os.path.join(test_dir, d)) for d in os.listdir(test_dir)])

if not train_images_exist or not test_images_exist:
    # Create data generators
    datagen = ImageDataGenerator(rescale=1./255)
    data_flow = datagen.flow_from_directory(base_dir, target_size=(150, 150), batch_size=32, class_mode='categorical', shuffle=True)

    # Extract file paths and labels
    file_paths = data_flow.filepaths
    labels = data_flow.classes

    # Extract class indices and create inverse mapping
    class_indices = data_flow.class_indices
    class_indices_inv = {v: k for k, v in class_indices.items()}

    # Perform train-test split
    train_paths, test_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.3, random_state=42, stratify=labels)

    # Save train and test images to respective directories
    def save_images(paths, labels, output_dir):
        """
        Save images to the specified output directory.
        """
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        for path, label in zip(paths, labels):
            label_folder = os.path.join(output_dir, class_indices_inv[label])
            if not os.path.exists(label_folder):
                os.makedirs(label_folder)
            shutil.copy(path, os.path.join(label_folder, os.path.basename(path)))

    save_images(train_paths, train_labels, train_dir)
    save_images(test_paths, test_labels, test_dir)
else:
    print("Train and test directories already contain images. Skipping data copy.")

# Handle unreadable folders
unreadable_folders = list_unreadable_folders(base_dir)

if unreadable_folders:
    print("Unreadable folders:")
    for folder in unreadable_folders:
        print(folder)
    
    resolved_folders = handle_unreadable_folders(unreadable_folders)
else:
    resolved_folders = []

# Copy additional readable folders to train and test directories
for folder_path in resolved_folders:
    # Determine if folder should be added to train or test directory
    add_to_train = input(f"Do you want to add '{folder_path}' to the train folder? (yes/no): ").strip().lower()
    if add_to_train == 'yes':
        copy_data(folder_path, train_dir)
    
    add_to_test = input(f"Do you want to add '{folder_path}' to the test folder? (yes/no): ").strip().lower()
    if add_to_test == 'yes':
        copy_data(folder_path, test_dir)

print("Process completed.")


Enter the path to the main dataset directory:  C:\Users\Abhishek\Desktop\project done\final_pest\rajesh_pest\pest\pest
Enter the path for the train folder:  D:\pest\train
Enter the path for the test folder:   D:\pest\test


Train and test directories already contain images. Skipping data copy.
Process completed.


In [3]:
# Data generators with data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load the images in batches
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

Found 29740 images belonging to 132 classes.
Found 12747 images belonging to 132 classes.


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Prepare data augmentation and generators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,  # Path to train directory
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Path to test directory
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Define num_classes dynamically based on the dataset
class_indices_inv = {v: k for k, v in train_generator.class_indices.items()}  # Inverse the class indices
num_classes = len(class_indices_inv)  # Use the length of class indices to get the number of classes

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Output layer with num_classes units
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=30,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    verbose=2
)


Found 29740 images belonging to 132 classes.
Found 12747 images belonging to 132 classes.


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


929/929 - 860s - 926ms/step - accuracy: 0.0902 - loss: 4.2502 - val_accuracy: 0.1138 - val_loss: 4.0165
Epoch 2/30
929/929 - 1s - 641us/step - accuracy: 0.0938 - loss: 4.1022 - val_accuracy: 0.0909 - val_loss: 4.0731
Epoch 3/30


C:\Users\Abhishek\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


929/929 - 524s - 564ms/step - accuracy: 0.1419 - loss: 3.8008 - val_accuracy: 0.1578 - val_loss: 3.7124
Epoch 4/30
929/929 - 1s - 658us/step - accuracy: 0.1875 - loss: 3.5828 - val_accuracy: 0.0909 - val_loss: 4.3231
Epoch 5/30
929/929 - 557s - 599ms/step - accuracy: 0.2087 - loss: 3.4291 - val_accuracy: 0.2077 - val_loss: 3.4723
Epoch 6/30
929/929 - 1s - 622us/step - accuracy: 0.2500 - loss: 3.3879 - val_accuracy: 0.0909 - val_loss: 4.1417
Epoch 7/30
929/929 - 543s - 585ms/step - accuracy: 0.2727 - loss: 3.0590 - val_accuracy: 0.2294 - val_loss: 3.3987
Epoch 8/30
929/929 - 1s - 727us/step - accuracy: 0.2188 - loss: 3.1261 - val_accuracy: 0.0909 - val_loss: 2.8768
Epoch 9/30
929/929 - 524s - 564ms/step - accuracy: 0.3580 - loss: 2.6267 - val_accuracy: 0.2254 - val_loss: 3.4916
Epoch 10/30
929/929 - 1s - 712us/step - accuracy: 0.1562 - loss: 3.0755 - val_accuracy: 0.3636 - val_loss: 3.0043
Epoch 11/30
929/929 - 552s - 594ms/step - accuracy: 0.4508 - loss: 2.1477 - val_accuracy: 0.2419 -

In [10]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Predict on test data
test_generator.reset()
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)

# Convert predictions to class indices
predicted_classes = np.argmax(predictions, axis=1)

# True classes
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


399/399 - 102s - 256ms/step - accuracy: 0.2090 - loss: 8.9396
Test Accuracy: 0.21
399/399 ━━━━━━━━━━━━━━━━━━━━ 101s 254ms/step
Classification Report:
                                 precision    recall  f1-score   support

                  Adristyrannus       0.03      0.02      0.03        45
       Aleurocanthus spiniferus       0.00      0.00      0.00        66
                    Ampelophaga       0.01      0.01      0.01       103
    Aphis citricola Vander Goot       0.00      0.00      0.00        47
               Apolygus lucorum       0.00      0.00      0.00        56
           Bactrocera tsuneonis       0.00      0.00      0.00        22
                Beet spot flies       0.00      0.00      0.00        40
                    Black hairy       0.00      0.00      0.00        93
   Brevipoalpus lewisi McGregor       0.00      0.00      0.00         7
             Ceroplastes rubens       0.00      0.00      0.00        32
           Chlumetia transversa       0.00    

In [30]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name from image
def predict_pest(image_path, model, class_labels):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)

    if confidence > 0.5:  # Set a confidence threshold for prediction
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name}")
    else:
        print("Sorry, I don't recognize the species.")

# Example usage
image_path = input("Enter the path to the pest image: ")
class_labels = list(test_generator.class_indices.keys())  # Get class labels from generator
predict_pest(image_path, model, class_labels)

Enter the path to the pest image:  C:\Users\Abhishek\Downloads\krzysztof-niewolny-pb5EBwMUc2w-unsplash.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Pest is found to be: Miridae


In [40]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name from image
def predict_pest(image_path, model, class_labels):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)

    if confidence > 0.5:  # Set a confidence threshold for prediction
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name}")
    else:
        print("Sorry, I don't recognize the species.")

# Example usage
image_path = input("Enter the path to the pest image: ")
class_labels = list(test_generator.class_indices.keys())  # Get class labels from generator
predict_pest(image_path, model, class_labels)

Enter the path to the pest image:  C:\Users\Abhishek\Desktop\project done\all pest\rajesh_pest\pest\pest\alfalfa weevil\30368.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
Sorry, I don't recognize the species.


In [36]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Define paths for training and validation data
train_path = input("Enter the path to the training dataset: ").strip()
test_path = input("Enter the path to the validation dataset: ").strip()

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Define the number of classes
num_classes = len(train_generator.class_indices)

# Load VGG16 with pre-trained ImageNet weights
vgg16 = VGG16(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

# Freeze the base layers of VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(vgg16.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout layer to prevent overfitting
x = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=vgg16.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0009),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Calculate class weights for imbalanced data
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00009, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=40,
    callbacks=[reduce_lr, early_stopping],
    class_weight=class_weights
)

# Save the model
model.save('vgg16_finetuned.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(validation_generator, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Predict on test data
validation_generator.reset()
predictions = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)

# Convert predictions to class indices
predicted_classes = np.argmax(predictions, axis=1)

# True classes
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


Enter the path to the training dataset:   D:\pest\train
Enter the path to the validation dataset:   D:\pest\test


Found 29740 images belonging to 132 classes.
Found 12747 images belonging to 132 classes.
Epoch 1/40


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


929/929 ━━━━━━━━━━━━━━━━━━━━ 3689s 4s/step - accuracy: 0.0105 - loss: 4.9744 - val_accuracy: 0.0185 - val_loss: 4.8592 - learning_rate: 9.0000e-04
Epoch 2/40
  1/929 ━━━━━━━━━━━━━━━━━━━━ 40:42 3s/step - accuracy: 0.0000e+00 - loss: 3.7546

C:\Users\Abhishek\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


929/929 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 3.7546 - val_accuracy: 0.0000e+00 - val_loss: 4.8745 - learning_rate: 9.0000e-04
Epoch 3/40
929/929 ━━━━━━━━━━━━━━━━━━━━ 3487s 4s/step - accuracy: 0.0117 - loss: 4.8299 - val_accuracy: 0.0414 - val_loss: 4.7894 - learning_rate: 9.0000e-04
Epoch 4/40
929/929 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.0000e+00 - loss: 5.2225 - val_accuracy: 0.0000e+00 - val_loss: 4.8855 - learning_rate: 9.0000e-04
Epoch 5/40
929/929 ━━━━━━━━━━━━━━━━━━━━ 5631s 6s/step - accuracy: 0.0491 - loss: 4.8435 - val_accuracy: 0.0537 - val_loss: 4.7977 - learning_rate: 9.0000e-04
Epoch 6/40
929/929 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0000e+00 - loss: 4.8552 - val_accuracy: 0.2727 - val_loss: 4.7858 - learning_rate: 9.0000e-04
Epoch 7/40
929/929 ━━━━━━━━━━━━━━━━━━━━ 3648s 4s/step - accuracy: 0.0153 - loss: 4.8786 - val_accuracy: 0.0587 - val_loss: 4.7668 - learning_rate: 9.0000e-04
Epoch 8/40
929/929 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/st

399/399 - 1046s - 3s/step - accuracy: 0.0573 - loss: 4.7637
Test Accuracy: 0.06
399/399 ━━━━━━━━━━━━━━━━━━━━ 1077s 3s/step
Classification Report:
                                 precision    recall  f1-score   support

                  Adristyrannus       0.00      0.00      0.00        45
       Aleurocanthus spiniferus       0.00      0.00      0.00        66
                    Ampelophaga       0.00      0.00      0.00       103
    Aphis citricola Vander Goot       0.00      0.00      0.00        47
               Apolygus lucorum       0.00      0.00      0.00        56
           Bactrocera tsuneonis       0.00      0.00      0.00        22
                Beet spot flies       0.00      0.00      0.00        40
                    Black hairy       0.00      0.00      0.00        93
   Brevipoalpus lewisi McGregor       0.00      0.00      0.00         7
             Ceroplastes rubens       0.00      0.00      0.00        32
           Chlumetia transversa       0.00      0.

C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [37]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=(150, 150)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name from image
def predict_pest(image_path, model, class_labels):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)

    if confidence > 0.5:  # Set a confidence threshold for prediction
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name}")
    else:
        print("Sorry, I don't recognize the species.")

# Example usage
image_path = input("Enter the path to the pest image: ")
class_labels = list(test_generator.class_indices.keys())  # Get class labels from generator
predict_pest(image_path, model, class_labels)

Enter the path to the pest image:  C:\Users\Abhishek\Desktop\project done\all pest\rajesh_pest\pest\pest\alfalfa weevil\30291.jpg


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 833ms/step
Sorry, I don't recognize the species.


In [42]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

# Define paths for training and validation data
train_path = input("Enter the path to the training dataset: ").strip()
test_path = input("Enter the path to the validation dataset: ").strip()

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Define the number of classes
num_classes = len(train_generator.class_indices)

# Load VGG16 with pre-trained ImageNet weights
vgg16 = VGG16(input_shape=(150, 150, 3), include_top=False, weights='imagenet')

# Freeze the base layers of VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(vgg16.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout layer to prevent overfitting
x = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=vgg16.input, outputs=x)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Calculate class weights for imbalanced data
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30,
    callbacks=[reduce_lr, early_stopping],
    class_weight=class_weights
)

# Save the model
model.save('vgg16_finetuned.h5')

# Evaluate the model
test_loss, test_accuracy = model.evaluate(validation_generator, verbose=2)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Predict on test data
validation_generator.reset()
predictions = model.predict(validation_generator, steps=validation_generator.samples // validation_generator.batch_size + 1)

# Convert predictions to class indices
predicted_classes = np.argmax(predictions, axis=1)

# True classes
true_classes = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

# Print classification report
print("Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


Enter the path to the training dataset:   D:\pest\train
Enter the path to the validation dataset:   D:\pest\test


Found 29740 images belonging to 132 classes.
Found 12747 images belonging to 132 classes.
Epoch 1/30


C:\Users\Abhishek\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


929/929 ━━━━━━━━━━━━━━━━━━━━ 3629s 4s/step - accuracy: 0.0143 - loss: 4.9260 - val_accuracy: 0.0616 - val_loss: 4.6192 - learning_rate: 1.0000e-04
Epoch 2/30
  1/929 ━━━━━━━━━━━━━━━━━━━━ 39:46 3s/step - accuracy: 0.0625 - loss: 4.1785

C:\Users\Abhishek\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


929/929 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.0625 - loss: 4.1785 - val_accuracy: 0.0000e+00 - val_loss: 4.7115 - learning_rate: 1.0000e-04
Epoch 3/30
929/929 ━━━━━━━━━━━━━━━━━━━━ 3853s 4s/step - accuracy: 0.0457 - loss: 4.6340 - val_accuracy: 0.1011 - val_loss: 4.3586 - learning_rate: 1.0000e-04
Epoch 4/30
929/929 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.0312 - loss: 5.5058 - val_accuracy: 0.1818 - val_loss: 4.4942 - learning_rate: 1.0000e-04
Epoch 5/30
929/929 ━━━━━━━━━━━━━━━━━━━━ 3565s 4s/step - accuracy: 0.0658 - loss: 4.4484 - val_accuracy: 0.1332 - val_loss: 4.1304 - learning_rate: 1.0000e-04
Epoch 6/30
929/929 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.1562 - loss: 4.5614 - val_accuracy: 0.0000e+00 - val_loss: 4.6476 - learning_rate: 1.0000e-04
Epoch 7/30
929/929 ━━━━━━━━━━━━━━━━━━━━ 3576s 4s/step - accuracy: 0.0853 - loss: 4.2553 - val_accuracy: 0.1381 - val_loss: 4.0434 - learning_rate: 1.0000e-04
Epoch 8/30
929/929 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accurac

399/399 - 1342s - 3s/step - accuracy: 0.1549 - loss: 3.8952
Test Accuracy: 0.15
399/399 ━━━━━━━━━━━━━━━━━━━━ 1103s 3s/step
Classification Report:
                                 precision    recall  f1-score   support

                  Adristyrannus       0.00      0.00      0.00        45
       Aleurocanthus spiniferus       0.00      0.00      0.00        66
                    Ampelophaga       0.01      0.01      0.01       103
    Aphis citricola Vander Goot       0.00      0.00      0.00        47
               Apolygus lucorum       0.00      0.00      0.00        56
           Bactrocera tsuneonis       0.00      0.00      0.00        22
                Beet spot flies       0.00      0.00      0.00        40
                    Black hairy       0.00      0.00      0.00        93
   Brevipoalpus lewisi McGregor       0.00      0.00      0.00         7
             Ceroplastes rubens       0.00      0.00      0.00        32
           Chlumetia transversa       0.00      0.

C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Abhishek\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [4]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array  # Updated import
import numpy as np

# Ask the user for the paths to the train, test data directories, and the image path
train_path = input("Enter the path to the training data directory: ")
test_path = input("Enter the path to the testing data directory: ")
image_path = input("Enter the path to the pest image: ")

# Define the image size and batch size
image_size = (150, 150)
batch_size = 32

# Define data generators for train and test datasets
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the train and test generators
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Retrieve the class labels from the train generator
class_labels = list(train_generator.class_indices.keys())

# Load the saved model
model = load_model('vgg16_finetuned.h5')

# Function to load and preprocess a single image
def load_and_preprocess_image(image_path, target_size=image_size):
    img = load_img(image_path, target_size=target_size)  # Updated function
    img_array = img_to_array(img)  # Updated function
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values
    return img_array

# Function to predict pest name from an image
def predict_pest(image_path, model, class_labels):
    img = load_and_preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = np.max(prediction)

    if confidence > 0.5:  # Set a confidence threshold for prediction
        pest_name = class_labels[predicted_class_index]
        print(f"Pest is found to be: {pest_name} with confidence {confidence:.2f}")
    else:
        print("Sorry, I don't recognize the species.")

# Predict pest based on user input
predict_pest(image_path, model, class_labels)


Enter the path to the training data directory:  D:\pest\train
Enter the path to the testing data directory:  D:\pest\test
Enter the path to the pest image:  C:\Users\Abhishek\Desktop\project done\all pest\rajesh_pest\pest\pest\alfalfa weevil\30368.jpg


Found 29740 images belonging to 132 classes.
Found 12747 images belonging to 132 classes.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Sorry, I don't recognize the species.


In [ ]:
# Save the entire model to a file
model.save('my_pest_classification_model.h5')
